原文地址： https://mp.weixin.qq.com/s?src=11&timestamp=1534251334&ver=1060&signature=tr-JnyoiOdoLZwYvqe-0DnOPcWPUEvdSSFpI-KWMnh1HfL3nf3xJUvlX0aa18mR-LdLBd3r5w1WbGMIdFinZ*2uT5d42y36BxfaZ8kxqNsEZaBMxOeMBszmoLqQOmpiB&new=1

故事的起因是上周六看《中国好声音》,一个周杰伦战队的学员用人工智能写的歌词，于是乎，我也有了这个想法，代码的主题思路是看Crossin先生的文章，虽然最后不能写出一首歌，但是押韵脚这事情分分钟搞定了

主题的思路，就是先抓取很多首歌曲的歌词，利用jieba分词后，将分好的词按照押韵表进行分类，最后匹配查询就可以了

## 准备一：押韵表

这个地方可以去网上搜押韵表

In [1]:
#引用各种需要的库
import requests
import jieba
import re
from xpinyin import Pinyin

In [2]:
p = Pinyin()

RhymeIndex = [('1', ['a', 'ia', 'ua']), ('2', ['ai', 'uai']), ('3', ['an', 'ian', 'uan']),
              ('4', ['ang', 'iang', 'uang']), ('5', ['ao', 'iao']), ('6', ['e', 'o', 'uo']), ('7', ['ei', 'ui']),
              ('8', ['en', 'in', 'un']), ('9', ['eng', 'ing', 'ong', 'iong']), ('10', ['er']), ('11', ['i']),
              ('12', ['ie', 'ye']), ('13', ['ou', 'iu']), ('14', ['u']), ('16', ['ue']), ('15', ['qu', 'xu', 'yu'])]

RhymeDct = {'ui': '7', 'uan': '3', 'ian': '3', 'iu': '13', 'en': '8', 'ue': '16', 'ing': '9', 'a': '1', 'ei': '7',
            'eng': '9', 'uo': '6', 'ye': '12', 'in': '8', 'ou': '13', 'ao': '5', 'uang': '4', 'ong': '9', 'ang': '4',
            'ai': '2', 'ua': '1', 'uai': '2', 'an': '3', 'iao': '5', 'ia': '1', 'ie': '12', 'iong': '9', 'i': '11',
            'er': '10', 'e': '6', 'u': '14', 'un': '8', 'iang': '4', 'o': '6', 'qu': '15', 'xu': '15', 'yu': '15'}

## 准备二：分词对应押韵表编码

分好的词与押韵表对应起来，举个栗子，比如“没有”对应的是“7-13”，就等于你给每个词都贴了一个标签，这样你以后想搜索的时候，就可以根据标签找到这些词了

In [3]:
def _analysis_words(words):
        word_py =p.get_pinyin((u'{}'.format(words)))
        lst_words = word_py.split('-')
        r = []
        for i in lst_words:
            while True:
                if not i:
                    break
                token = RhymeDct.get(i, None)
                if token:
                    r.append(token)
                    break
                i = i[1:]
        if len(r) == len(words):
            return '-'.join(r)

## 第一步：爬虫抓取歌词信息

这个地方数据爬取的越多，肯定你的词库就越壮大，后面分词也越高，我这里只爬取了3首歌曲的歌词，并且最后是存储到txt中，当然，放数据库里就更好了

In [4]:
def GetKeyword():
    #歌曲列表
    # url = 'http://music.163.com/api/playlist/detail?id=808976784'
    # req = requests.get(url)
    # data = req.json()
    # print(data['result']['tracks'] )
    # tracks =data['result']['tracks']  #歌曲列表
    tracks=["431795900",'33850315','430053482']
    #写入记事本文件
    with open('keyword.txt','a') as f:
        f.write("[")
        for i in tracks:
            print(111)
            #歌词
            # lrcurl = "http://music.163.com/api/song/lyric?os=pc&id="+str(i['id'])+"&lv=-1&kv=-1&tv=-1"
            lrcurl = "http://music.163.com/api/song/lyric?os=pc&id="+str(i)+"&lv=-1&kv=-1&tv=-1"
            lrcreq = requests.get(lrcurl)
            dt = lrcreq.json()
            lrc=re.sub(u"\\[.*?]", "", dt['lrc']['lyric'])
            #jieba分词
            seg_list = list(jieba.cut(lrc, cut_all=True))
            for i in seg_list:
                #加入判断，只写入2个字组成的词
                if len(i)==2:
                    #写入格式：{'7-13':'追求'}
                    if _analysis_words(i)!=None:
                        f.write("{'"+_analysis_words(i)+"':'"+i+"'},")
        f.write("]")
        f.close()

## 第二步：调用分词的方法

In [5]:
GetKeyword()

111


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\CHENJI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.354 seconds.
Prefix dict has been built succesfully.


111
111


## 第三步：分析分词后的txt

In [6]:
def Findkey(str):
    result={}
    with open('keyword.txt', 'r') as f:
        # print(f.readlines())
        list=eval(f.readlines()[0])
        for item in list:
            if item.get(str):
                key=item.get(str)
                number=result.get(key)
                #如果一个词出现多次，进行次数累加，用来表示频次
                if number !=None and number>=1:
                    result[key]=number+1
                else:
                    result.update({key:1})
        f.close()
        print(result)

## 第四步：程序入口

In [8]:
key=input("请输入关键词:")
str=_analysis_words(key)
print("匹配押韵的词：")
Findkey(str)

请输入关键词:妮妮
匹配押韵的词：
{'只是': 3, '一时': 1, '一直': 3, '实际': 1, '实意': 1, '吸气': 1, '里子': 1, '第一': 1, '细皮': 1, '七里': 1, '袭击': 1, '之一': 1, '仔细': 1}


## 第五步：创作自己的FreeStyle